In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq 
llm= ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm 

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000209362CEFE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000209363003D0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
import os
from langchain.document_loaders import PyPDFLoader

file_path = "C:/Users/goyal/Desktop/project/sewak_ai/data.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()
print("Processed all PDFs.")


Processed all PDFs.


In [4]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\goyal\Desktop\project\sewak_ai\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=80)
splits = text_splitter.split_documents(data)

In [7]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000209717FF850>, search_kwargs={})

In [8]:
# question = "what is the constitution of india  ?"
# docs = vectorstore.similarity_search(question, k=4)  # Pass the query string directly

# # Print results
# print(f"Found {len(docs)} results:")
# for doc in docs:
#     print(doc)

In [9]:
from langchain_core.prompts import ChatPromptTemplate

# Create refined RAG prompt
rag_prompt = (
    "You are Sewak AI, an AI assistant for the constitution of India, specializing in queries related to "
    "Articles details"
    "Explaining constituion in easy language"
    "Do not express personal opinions. Present constitutional provisions as they are, including different interpretations when necessary."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",rag_prompt),
        ("human","{input}"),
    ]
)

In [10]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [11]:
response=rag_chain.invoke({"input":"What is constitution of india"})
response

{'input': 'What is constitution of india',
 'context': [Document(metadata={'page': 3, 'page_label': '4', 'source': 'C:/Users/goyal/Desktop/project/sewak_ai/data.pdf'}, page_content='THE CONSTITUTION OF INDIA   \n____________                                                                     \n                             \nCONTENTS \n__________'),
  Document(metadata={'page': 126, 'page_label': '127', 'source': 'C:/Users/goyal/Desktop/project/sewak_ai/data.pdf'}, page_content='THE CONSTITUTION OF  INDIA \n(Part VI.—The States) \n96'),
  Document(metadata={'page': 139, 'page_label': '140', 'source': 'C:/Users/goyal/Desktop/project/sewak_ai/data.pdf'}, page_content='THE CONSTITUTION OF  INDIA \n(Part VI.—The States) \n109'),
  Document(metadata={'page': 200, 'page_label': '201', 'source': 'C:/Users/goyal/Desktop/project/sewak_ai/data.pdf'}, page_content='this Constitution, or for the purposes of any enactment relating to the \nGovernment of India heretofore in force, nor shall any perso

In [12]:
response['answer']

'The Constitution of India is the supreme law of India. \n\nIt lays down the framework defining the fundamental political principles on which India is governed and outlines the powers and duties of the government established by it. \n\nYou\'ve provided excerpts from Part VI of the Constitution, which deals with "The States." To understand the Constitution better, I recommend exploring each part and article in detail.  \n\nWould you like me to explain any specific article or part of the Constitution in simpler terms? \n\n'

In [13]:
response2=rag_chain.invoke({"input":"What are the article for discrimination"})

In [14]:
response2['answer']

"The Indian Constitution prohibits discrimination in several articles. Here are the key ones you mentioned:\n\n* **Article 15:** This article states that the State shall not discriminate against any citizen on grounds of religion, race, caste, sex, or place of birth. \n\n* **Article 29:** This article protects the interests of minorities. It states that any law made by the Parliament or a State Legislature shall not discriminate against any citizen on the ground of religion, race, caste, language or any of them.\n\n* **Article 30:** This article guarantees the right of minorities to establish and administer educational institutions of their choice.\n\n**Important Note:**\n\nWhile these articles prohibit discrimination, Article 15(3) and 15(4) allow the state to make special provisions for the advancement of any socially and educationally backward classes or for the Scheduled Castes and the Scheduled Tribes. \n\n\nIt's important to remember that the interpretation of these articles has 

### Adding chat history

In [23]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question"
    "which might reference context in the chat history"
    "formulate a standalone question which can be understood"
    "without the chat history.DO not answeer the question,"
    "just reformulate it if needed and outherwisw return it as is."
)
contextualize_q_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [25]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000209717FF850>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag

In [27]:
qa_prompt= ChatPromptTemplate.from_messages(
    [
        ("system",rag_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [28]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [30]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What are the article for discrimination in the constituton of india"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="What are the punishment for it?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2["answer"])

You're asking about articles that address discrimination in the Indian Constitution.  

While the Constitution doesn't have a single article explicitly titled "Discrimination,"  several articles prohibit it in various contexts.  Here are some key ones:

**Fundamental Rights (Part III)**

* **Article 14: Equality Before Law** - This is the cornerstone of equality. It states that the State shall not deny to any person equality before the law or the equal protection of the laws within the territory of India.
* **Article 15: Prohibition of Discrimination on Grounds of Religion, Race, Caste, Sex or Place of Birth** - As discussed earlier, this article specifically prohibits discrimination based on these grounds.
* **Article 16: Equality of Opportunity in Matters of Public Employment** - This article ensures that there shall be equality of opportunity in matters of public employment. It prohibits discrimination based on religion, race, caste, sex, descent, place of birth, or any of them.

**

In [31]:
chat_history

[HumanMessage(content='What are the article for discrimination in the constituton of india', additional_kwargs={}, response_metadata={}),
 AIMessage(content='You are asking about articles related to discrimination in the Indian Constitution. \n\nHere are the articles you mentioned and their relevant provisions:\n\n**Article 15: Prohibition of Discrimination on Grounds of Religion, Race, Caste, Sex or Place of Birth**\n\n* **(1)** The State shall not discriminate against any citizen on grounds only of religion, race, caste, sex, place of birth or any of them.\n\n* **(2) and (3)** These clauses permit the State to make special provisions for women and children, and for the advancement of any socially or educationally backward classes of citizens.\n\n* **(4)** This clause allows the State to make any law providing for the custody, adoption or guardianship of infants, for the purpose of their wellbeing.\n\n**Article 29: Protection of Interests of Minorities**\n\n* **(1)** Any section of th

In [33]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",

)

In [34]:
conversational_rag_chain.invoke(
    {"input": "what is right to education"},
    config={
        "configurable":{"session_id":"abc123"}
    },
)["answer"]

'According to the Indian Constitution, **Article 21A** guarantees the **Right to Education** for all children between the ages of six and fourteen years. \n\nThe State is obligated to provide **free and compulsory education** to this age group. This means that education is mandatory and cannot be denied to any child within this age range, and it must be provided without any cost to the parents.\n\n**Article 30** further elaborates on the rights of minorities. It states that all minorities, whether based on religion or language, have the right to establish and administer their own educational institutions. \n\n\nLet me know if you have any other questions about these articles or any other provisions of the Indian Constitution. \n\n'